# **Flood Analysis for Valencia, Spain**

This code performs a flood analysis using Sentinel-2 and Sentinel-1 data over Valencia, Spain. It includes:
- Cloud masking
- Histogram matching
- Pansharpening
- Resampling
- Calculation of NDVI, NDWI, and NDBI indices
- Visualization of results on a map
- Export of results as GeoTIFF files.

## **1: Import Libraries and Define Constants**

## *Explanation of each library and module*

1. **`os`**:
   - A standard Python module for interacting with the operating system.
   - Used for file and directory manipulation, environment variable access, and path operations.
   
2. **`ee`** (Earth Engine):
   - A module for accessing Google Earth Engine (GEE) in Python.
   - GEE is a planetary-scale geospatial data analysis platform.
     
3. **`geemap`**:
   - A Python library for interactive mapping and geospatial analysis with Google Earth Engine (GEE).
   - It provides tools for analyzing satellite imagery and integrating GEE data into Python.

In [1]:
import os
import geemap
import ee

# Constants

OUT_PATH = 'Output_Results'
PRE_FLOOD_RANGE = ('2024-08-25', '2024-09-25')
POST_FLOOD_RANGE = ('2024-09-26', '2024-11-25')

VIS_RGB = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
VIS_PARAMS = {
    'NDVI_DIFF': {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'green']},
    'NDWI_DIFF': {'min': -0.5, 'max': 0.5, 'palette': ['blue', 'white', 'cyan']},
    'NDBI_DIFF': {'min': -0.5, 'max': 0.5, 'palette': ['gray', 'white', 'black']}
}

NDVI_THRESHOLD = 0.001
NDWI_THRESHOLD = 0.001
NDBI_THRESHOLD = 0.001

## **2: Define Utility Functions**

### *Authenticate and initialize Google Earth Engine*

In [2]:
def initialize_earth_engine():
    
    ee.Authenticate()
    ee.Initialize()

### *Create output directory if it doesn't exist*

In [3]:
def create_output_directory():
    
    if not os.path.exists(OUT_PATH):
        os.mkdir(OUT_PATH)

### *Define the Region of Interest (ROI) around Valencia, Spain*

In [4]:
def define_roi():
    return ee.Geometry.Polygon([
    [[-0.392, 39.518], [-0.392, 39.390], [-0.297, 39.390], [-0.297, 39.518], [-0.392, 39.518]]
])

### *Add a Bounding Box layer to the map*

In [5]:
def add_bounding_box_to_map(map_object, roi, color="blue", name="Bounding Box"):
   
    bounding_box_layer = ee.FeatureCollection([ee.Feature(roi)])
    map_object.addLayer(bounding_box_layer.style(
        color=color, width=2, fillColor=None
    ), {}, name)

### *Mask Clouds from Sentinel-2 images*

In [6]:
def mask_sentinel2_clouds(image):
    
    scl = image.select('SCL')  # Scene classification band
    cloud_free = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return image.updateMask(cloud_free)

### *Perform simple mean Pansharpening*

In [7]:
def pansharpen_simple_mean(multispectral, panchromatic, bands):
   
    multispectral_mean = multispectral.reduce(ee.Reducer.mean())
    pansharpened_bands = [
        multispectral.select([band]).add(panchromatic.subtract(multispectral_mean))
        for band in bands
    ]
    return ee.Image.cat(pansharpened_bands).rename(bands)

### *Resample the image to a specific resolution*

In [8]:
def resample_image(image, scale, resampling_method='nearest'):
    
    return image.resample(resampling_method).reproject(
        crs=image.projection(),
        scale=scale
    )

### *Perform histogram matching for pre-flood and post-flood images*

In [9]:
def histogram_match_simple(pre_flood_image, post_flood_image, band_name, region, scale):
   
    def normalize(image, band_name):
        stats = image.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=region,
            scale=scale,
            bestEffort=True
        )
        min_val = ee.Number(stats.get(f"{band_name}_min"))
        max_val = ee.Number(stats.get(f"{band_name}_max"))
        return image.select(band_name).subtract(min_val).divide(max_val.subtract(min_val))

    pre_flood_normalized = normalize(pre_flood_image, band_name)
    post_flood_normalized = normalize(post_flood_image, band_name)

    post_stats = post_flood_image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=region,
        scale=scale,
        bestEffort=True
    )
    post_min = ee.Number(post_stats.get(f"{band_name}_min"))
    post_max = ee.Number(post_stats.get(f"{band_name}_max"))

    matched = pre_flood_normalized.multiply(post_max.subtract(post_min)).add(post_min)
    return matched.rename(f"{band_name}_matched")

### *Calculate the area of regions in an image above a given threshold for a specific index*

In [10]:
def calculate_area(image, roi, threshold=0):
    # Mask the image based on the threshold
    significant_area = image.updateMask(image.gt(threshold))
    
    # Calculate the area in square meters
    pixel_area = significant_area.multiply(ee.Image.pixelArea())
    area_stats = pixel_area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=10,
        maxPixels=1e13
    )
    
    # Convert area_stats to a Python dictionary
    area_stats_dict = area_stats.getInfo()
    
    # Get the band name dynamically
    band_name = image.bandNames().get(0).getInfo()
    
    # Check if the band name exists in the dictionary
    if band_name not in area_stats_dict:
        print(f"Warning: Band '{band_name}' not found in area_stats: {area_stats_dict}")
        return 0  # Return 0 if no data is found
    
    # Convert to square kilometers
    area_km2 = area_stats_dict[band_name] / 1e6
    return area_km2

### *Calculate NDVI, NDWI, and NDBI indices*

In [11]:
def calculate_indices(image):
   
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')  # NIR and RED
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')  # GREEN and NIR
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')  # SWIR and NIR
    return image.addBands([ndvi, ndwi, ndbi])

### *Export Images* - TIFF format

In [12]:
def export_image_to_tiff(image, file_name, region, scale=10):
    
    task = ee.batch.Export.image.toDrive(
        image=image.clip(region),
        description=file_name,
        fileNamePrefix=file_name,
        folder=OUT_PATH,
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    print(f"Export task started for {file_name}. Check your Google Drive for the exported TIFF file.")

## **2: Define Utility Functions**

**Main workflow for flood analysis in Valencia, Spain:**
- Authenticate and initialize Earth Engine.
- Define a bounding box for Valencia.
- Preprocess Sentinel-2 and Sentinel-1 data.
- Apply histogram matching, pansharpening, and resampling.
- Calculate NDVI, NDWI, and NDBI indices.
- Visualize results on a map and export GeoTIFF files.
    

In [14]:
def main():

    # Initialize Earth Engine and create output directory
    initialize_earth_engine()
    create_output_directory()

    # Define region of interest (ROI)
    ROI = define_roi()

    # Calculate the total area of the ROI
    total_area = ROI.area().divide(1e6).getInfo()  # Convert from m² to km²
    print("Total Area of ROI (km²):", total_area)
    
    # Set up base map
    base_map = geemap.Map(center=[39.47, -0.38], zoom=12)  # Centered on Valencia
    add_bounding_box_to_map(base_map, ROI, color="blue", name="Valencia Bounding Box")

    # Preprocess Sentinel-2 data
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(ROI) \
        .filterDate(PRE_FLOOD_RANGE[0], POST_FLOOD_RANGE[1]) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))

    pre_flood = sentinel2.filterDate(PRE_FLOOD_RANGE[0], PRE_FLOOD_RANGE[1]).map(mask_sentinel2_clouds)
    post_flood = sentinel2.filterDate(PRE_FLOOD_RANGE[0], POST_FLOOD_RANGE[1]).map(mask_sentinel2_clouds)

    # Calculate median images for pre- and post-flood
    pre_flood_median = pre_flood.median()
    post_flood_median = post_flood.median()

    # Apply histogram matching and pansharpening
    pre_flood_matched_b8 = histogram_match_simple(pre_flood_median, post_flood_median, 'B8', ROI, 10)
    pre_flood_matched = pre_flood_median.addBands(pre_flood_matched_b8)

    multispectral = pre_flood_matched.select(['B4', 'B3', 'B2'])  # RGB bands
    panchromatic = pre_flood_matched.select('B8_matched')  # Histogram-matched B8 band
    pansharpened_image = pansharpen_simple_mean(multispectral, panchromatic, ['B4', 'B3', 'B2'])

    # Calculate indices (NDVI, NDWI, NDBI)
    pre_flood_indices = calculate_indices(pre_flood_median)
    post_flood_indices = calculate_indices(post_flood_median)

    # Extract individual indices
    pre_ndvi = pre_flood_indices.select('NDVI')
    post_ndvi = post_flood_indices.select('NDVI')
    pre_ndwi = pre_flood_indices.select('NDWI')
    post_ndwi = post_flood_indices.select('NDWI')
    pre_ndbi = pre_flood_indices.select('NDBI')
    post_ndbi = post_flood_indices.select('NDBI')

    # Calculate differences
    ndvi_diff = post_ndvi.subtract(pre_ndvi)
    ndwi_diff = post_ndwi.subtract(pre_ndwi)
    ndbi_diff = post_ndbi.subtract(pre_ndbi)

    # Calculate affected areas for pre- and post-flood indices
    pre_ndvi_area = calculate_area(pre_ndvi, ROI, NDVI_THRESHOLD)
    post_ndvi_area = calculate_area(post_ndvi, ROI, NDVI_THRESHOLD)
    
    pre_ndwi_area = calculate_area(pre_ndwi, ROI, NDWI_THRESHOLD)
    post_ndwi_area = calculate_area(post_ndwi, ROI, NDWI_THRESHOLD)
    
    pre_ndbi_area = calculate_area(pre_ndbi, ROI, NDBI_THRESHOLD)
    post_ndbi_area = calculate_area(post_ndbi, ROI, NDBI_THRESHOLD)
    
    # Compute the difference in areas
    ndvi_area_change = post_ndvi_area - pre_ndvi_area
    ndwi_area_change = post_ndwi_area - pre_ndwi_area
    ndbi_area_change = post_ndbi_area - pre_ndbi_area

    ndvi_percentage_change = (ndvi_area_change / total_area) * 100
    ndwi_percentage_change = (ndwi_area_change / total_area) * 100
    ndbi_percentage_change = (ndbi_area_change / total_area) * 100
    
    # Print the results
    print('NDVI Pre-Flood Area (km²):', pre_ndvi_area)
    print('NDVI Post-Flood Area (km²):', post_ndvi_area)
    print('NDVI Area Change (km²):', ndvi_area_change)
    
    print('NDWI Pre-Flood Area (km²):', pre_ndwi_area)
    print('NDWI Post-Flood Area (km²):', post_ndwi_area)
    print('NDWI Area Change (km²):', ndwi_area_change)
    
    print('NDBI Pre-Flood Area (km²):', pre_ndbi_area)
    print('NDBI Post-Flood Area (km²):', post_ndbi_area)
    print('NDBI Area Change (km²):', ndbi_area_change)

    print('NDBI Pre-Flood Area (km²):', pre_ndbi_area)
    print('NDBI Post-Flood Area (km²):', post_ndbi_area)
    print('NDBI Area Change (km²):', ndbi_area_change)
    print('NDBI Percentage Change (%):', ndbi_percentage_change)


    # Create NDWI increase layer (to highlight flood-affected areas)
    ndwi_increase = ndwi_diff.updateMask(ndwi_diff.gt(0))

    # Calculate the total area of the ROI
    total_area = ROI.area().divide(1e6).getInfo()  # Convert from m² to km²
    print("Total Area of ROI (km²):", total_area)

    # DEM-Based Flood Risk Analysis
    dem = ee.Image("USGS/SRTMGL1_003").select([0]).clip(ROI)  # Ensure single band and clip to ROI

    # Manually create Flood Risk Map
    low_risk = dem.gt(50)  # Elevation above 50m
    medium_risk = dem.lte(50).And(dem.gt(10))  # Elevation 10m–50m
    high_risk = dem.lte(10)  # Elevation below or equal to 10m

    flood_risk_vis = low_risk.multiply(1).add(medium_risk.multiply(2)).add(high_risk.multiply(3))

    # Visualization parameters
    dem_vis_params = {'min': 0, 'max': 100, 'palette': ['white', 'gray', 'black']}
    flood_risk_vis_params = {'min': 1, 'max': 3, 'palette': ['green', 'yellow', 'red']}

    # Add layers to the base map
    base_map.addLayer(pre_flood_median.clip(ROI), VIS_RGB, 'Pre-Flood Imagery')
    base_map.addLayer(post_flood_median.clip(ROI), VIS_RGB, 'Post-Flood Imagery')
    base_map.addLayer(pansharpened_image.clip(ROI), VIS_RGB, 'Pansharpened Image')
    
    base_map.addLayer(ndvi_diff.clip(ROI), VIS_PARAMS['NDVI_DIFF'], 'NDVI Difference')
    base_map.addLayer(ndwi_diff.clip(ROI), VIS_PARAMS['NDWI_DIFF'], 'NDWI Difference')
    base_map.addLayer(ndbi_diff.clip(ROI), VIS_PARAMS['NDBI_DIFF'], 'NDBI Difference')

    base_map.addLayer(ndwi_increase.clip(ROI), {'min': 0, 'max': 1, 'palette': ['blue']}, 'NDWI Increase (Flooded Areas)')
    base_map.addLayer(dem.clip(ROI), dem_vis_params, "DEM Elevation")
    base_map.addLayer(flood_risk_vis.clip(ROI), flood_risk_vis_params, "Flood Risk (Based on DEM)")

    # Export GeoTIFFs for all layers
    export_image_to_tiff(pre_flood_median, "Pre_Flood_Imagery", ROI)
    export_image_to_tiff(post_flood_median, "Post_Flood_Imagery", ROI)
    export_image_to_tiff(pansharpened_image, "Pansharpened_Image", ROI)
    export_image_to_tiff(ndvi_diff, "NDVI_Difference", ROI)
    export_image_to_tiff(ndwi_diff, "NDWI_Difference", ROI)
    export_image_to_tiff(ndbi_diff, "NDBI_Difference", ROI)
    export_image_to_tiff(ndwi_increase, "NDWI_Increase", ROI)
   
    export_image_to_tiff(flood_risk_vis.clip(ROI), "Flood_Risk_Map", ROI)

    print("All export tasks have been started. Check your output directory.")

    # Return the base map for visualization
    return base_map


# Run the main function and display the base map
base_map = main()
base_map  # Displays the interactive map

Total Area of ROI (km²): 116.09075431508244
NDVI Pre-Flood Area (km²): 14.35534104274565
NDVI Post-Flood Area (km²): 14.038524363809618
NDVI Area Change (km²): -0.31681667893603205
NDWI Pre-Flood Area (km²): 4.240517761966586
NDWI Post-Flood Area (km²): 4.178728272657948
NDWI Area Change (km²): -0.061789489308638146
NDBI Pre-Flood Area (km²): 5.678620420769274
NDBI Post-Flood Area (km²): 4.780648899508967
NDBI Area Change (km²): -0.897971521260307
NDBI Pre-Flood Area (km²): 5.678620420769274
NDBI Post-Flood Area (km²): 4.780648899508967
NDBI Area Change (km²): -0.897971521260307
NDBI Percentage Change (%): -0.7735082148084924
Total Area of ROI (km²): 116.09075431508244
Export task started for Pre_Flood_Imagery. Check your Google Drive for the exported TIFF file.
Export task started for Post_Flood_Imagery. Check your Google Drive for the exported TIFF file.
Export task started for Pansharpened_Image. Check your Google Drive for the exported TIFF file.
Export task started for NDVI_Differ

Map(center=[39.47, -0.38], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…